# Material de apoyo

In [29]:
import pymongo
import re
import pandas as pd
import numpy as np
import itertools

MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [3]:
cursor = db.companies.find( { 
                            "$and": [
                                    {"$or": [ 
                                            { "category_code": "software" }, 
                                            { "category_code": "web" },
                                            { "category_code": "consulting" },
                                            { "category_code": "public_relations" }
                                    ]},
                                    { "founded_year": {"$gte": 2010} },
                                    { "deadpooled_year": None},
                                    { "$or": [
                                            {"number_of_employees": {"$lte": 100}},
                                            {"number_of_employees": {"$gte": 1000}}
                                    ] },
                                    { "$and": [
                                            {"offices.latitude": {"$ne": None}},
                                            {"offices.longitude": {"$ne": None}}
                            ] } ] } )
#print(list(map(lambda r: convertCompany(r),cursor)))
result = list(map(lambda r: [r["name"], len(r["offices"])],list(cursor)))
print(result)

[['Fixya', 1], ['Whisper Labs', 0], ['CollegeConvo', 1], ['Carfeine', 1], ['Ziippi', 1], ['JumpForward', 1], ['Bling Easy', 0], ['Hany', 0], ['Navara', 1], ['PriceHub', 0], ['FamilyDen', 0], ['Yipit', 1], ['GroupTable', 0], ['Clowdy', 1], ['TwitBacks', 0], ['Unison Technologies', 1], ['Reachoo', 1], ['eCampusCash', 1], ['troovy', 0], ['FAT Media', 0], ['kinDragon', 1], ['zahdoo', 0], ['CompareChecker', 0], ['KarmaKorn', 0], ['Pitchie', 0]]


In [12]:
cursor = db.companies.find( { 
                            "$and": [
                                    {"$or": [ 
                                            { "category_code": "software" }, 
                                            { "category_code": "web" },
                                            { "category_code": "consulting" },
                                            { "category_code": "public_relations" }
                                    ]},
                                    { "founded_year": {"$gte": 2008} },
                                    { "deadpooled_year": None},
                                    { "$or": [
                                            { "number_of_employees": {"$lte": 100} },
                                            { "number_of_employees": {"$gte": 1000} }
                                    ] },
                                    { "offices": { "$exists": True, "$ne": [] } }
                            ] } )
#print(list(map(lambda r: convertCompany(r),cursor)))
result = list(map(lambda r: [r["name"], len(r["offices"])],list(cursor)))
print(result)

[['GoingOn', 1], ['headr', 2], ['Fixya', 1], ['First30Days', 1], ['PokerPlayersSocialNetwork', 1], ['thatsmymouse', 1], ['Tvosz', 1], ['Unknown Vector', 1], ['Poodz', 1], ['HealthcareMagic', 1], ['Stumpedia', 1], ['Newspond', 1], ['Scred', 1], ['Particle', 1], ['ZapNat', 1], ['nobosh', 1], ['Jpari', 1], ['Simplicant', 1], ['Wordki', 1], ['CreateDebate', 1], ['ReadBurner', 1], ['InformedMD', 1], ['Intent', 1], ['BuyerRaja', 1], ['Weplay', 1], ['Kalibera', 1], ['Woopra', 1], ['Noovo', 1], ['Recommend Box', 1], ['Learn10', 2], ['Weardrobe', 1], ['Cotendo', 2], ['GamerNook', 1], ['Twubble', 1], ['Edistorm', 2], ['FinFolio', 1], ['FragMob', 1], ['skoogO', 1], ['Jobingenieur', 1], ['Cutcaster', 1], ['TweetMeme', 1], ['Mad Mimi', 1], ['Beer Universe', 1], ['Macroaxis', 1], ['EQAL', 1], ['All-rankings', 1], ['Mollom', 1], ['Crowdstatus', 1], ['Imagine Whatever', 1], ['Sideline Report', 1], ['WotsCool', 1], ['UserVoice', 2], ['The Publicity Wheel', 1], ['New Relic', 1], ['Audana', 1], ['Tensor 

In [37]:
#def toGeoJSON(array):
#    lista = []
 #   for i in range(len(array)):
 #       lista.append( {
  #          "type":"Point",
   #         "coordinates":[(array[i]["latitude"]), (array[i]["longitude"])]
   #     })
 #   return lista

def convertCompany(company):
    lista = []
    for i in range(len(company["offices"])):
        lista.append( {
            "name": company["name"],
            "sector":company["category_code"],
            "twitter_account": company["twitter_username"],
            "position": (company["offices"][i]["latitude"], company["offices"][i]["longitude"])
        })
    #merged = list(itertools.chain(*lista))
    return lista

cursor = db.companies.find( { 
                            "$and": [
                                    {"$or": [ 
                                            { "category_code": "software" }, 
                                            { "category_code": "web" },
                                            { "category_code": "consulting" },
                                            { "category_code": "public_relations" }
                                    ]},
                                    { "founded_year": {"$gte": 2008} },
                                    { "deadpooled_year": None},
                                    { "$or": [
                                            { "number_of_employees": {"$lte": 100} },
                                            { "number_of_employees": {"$gte": 1000} }
                                    ] },
                                    { "offices": { "$exists": True, "$ne": [] } }
                            ] } )
geoCompanies = list(map(lambda r: convertCompany(r),cursor))
geoCompanies[0]


[{'name': 'GoingOn',
  'sector': 'software',
  'twitter_account': 'GoingOn_Network',
  'position': (37.782263, -122.392142)}]

In [89]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[(array["longitude"]), (array["latitude"])]
    }

def convertCompany(company):
    posicion = []
    for i in range(len(company["offices"])):
        if company["offices"][i]["longitude"] is None or company["offices"][i]["latitude"] is None:
            pass
        else:
            posicion.append( {
                "name": company["name"],
                "sector":company["category_code"],
                "twitter_account": company["twitter_username"],
                "position": toGeoJSON(company["offices"][i])
            } )
    return posicion

def flatten(posicion):
    plano = []
    for j in range(len(posicion)):
        for i in range(len(posicion[j])):
            plano.append(posicion[j][i])
    return plano

def toDataFrame(plano):
    df = pd.DataFrame(plano)
    return df

cursor = db.companies.find( { 
                            "$and": [
                                    {"$or": [ 
                                            { "category_code": "software" }, 
                                            { "category_code": "web" },
                                            { "category_code": "consulting" },
                                            { "category_code": "public_relations" }
                                    ]},
                                    { "founded_year": {"$gte": 2008} },
                                    { "deadpooled_year": None},
                                    { "$or": [
                                            { "number_of_employees": {"$lte": 100} },
                                            { "number_of_employees": {"$gte": 1000} }
                                    ] },
                                    { "offices": { "$exists": True, "$ne": [] } }
                            ] } )

geoCompanies = list(map(lambda r: convertCompany(r),cursor))

plano = flatten(geoCompanies)

data = toDataFrame(plano)

data.head()

,name,position,sector,twitter_account
0,GoingOn,"{'type': 'Point', 'coordinates': [-122.392142,...",software,GoingOn_Network
1,headr,"{'type': 'Point', 'coordinates': [13.4109071, ...",web,tripodsocial
2,Fixya,"{'type': 'Point', 'coordinates': [-122.323895,...",web,MUaliyu
3,First30Days,"{'type': 'Point', 'coordinates': [-85.717393, ...",web,clickariane
4,PokerPlayersSocialNetwork,"{'type': 'Point', 'coordinates': [-122.187925,...",web,


In [99]:
data.loc[data["name"]=="WJT Global Solutions"]["position"]

286    {'type': 'Point', 'coordinates': [-73.821321, ...
287    {'type': 'Point', 'coordinates': [-70.566466, ...
288    {'type': 'Point', 'coordinates': [88.363895, 2...
289    {'type': 'Point', 'coordinates': [121.475916, ...
290    {'type': 'Point', 'coordinates': [77.22496, 28...
Name: position, dtype: object

In [ ]:
API_KEY = "XXXX"
API_SECRET = "XXXX"
ACCESS_TOKEN = "XXXX"
ACCESS_TOKEN_SECRET = "XXXX"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
def getFollowers(twitteruser):
    followers = []
    for tweet in twitteruser:
        if twitteruser is None:
            followers.append(0)
        else:
            try:
                user = api.get_user(tweet)
                followers.append(user.followers_count)
            except Exception as e:
                if e == "[{u'message': u'Rate limit exceeded', u'code': 88}]":
                    time.sleep(60*15)
                else:
                    followers.append("invalid username")
    followers = pd.DataFrame(followers)
    return followers


def joinData2(data1, data2):
    data1["followers"] = data2[0]
    data_clean = data1.sort_values(by=["count"], ascending=False)
    return data_clean


followers = getFollowers(data_clean["twitter_account"])

data_clean = joinData2(data_clean, followers)
data_clean.head(10)

In [ ]:
followers = []

for tweet in data_clean["twitter_account"]:
    if data_clean["twitter_account"] is None:
        followers.append(0)
    else:
        try:
            user = api.get_user(tweet)
            followers.append(user.followers_count)
        except Exception as e:
            if e == "[{u'message': u'Rate limit exceeded', u'code': 88}]":
                time.sleep(60*15)
            else:
                followers.append("invalid username")